## Asynchronous scraping

In [1]:
import aiohttp
import asyncio
from bs4 import BeautifulSoup
import csv
import re

In [3]:
!pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

DEPRECATION: Loading egg at c:\users\photo\anaconda3\lib\site-packages\learntools-0.3.4-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [10]:
async def scrape_and_save_links(text):
    soup = BeautifulSoup(text, 'html.parser')
    file = open('csv_file', 'a', newline='')
    writer = csv.writer(file, delimiter=',')

    for link in soup.findAll('a', attrs={'href': re.compile('^http')}):
        link = link.get('href')
        writer.writerow([link])
    file.close()

In [11]:
async def fetch(session, url):
    try:
        async with session.get(url) as response:
            text = await response.text()
            task = asyncio.create_task(scrape_and_save_links(text))
            await task
    except Exception as e:
        print(str(e))

In [12]:
async def scrape(urls):
    tasks = []
    async with aiohttp.ClientSession() as session:
        for url in urls:
            tasks.append(fetch(session,url))
        await asyncio.gather(*tasks)

In [13]:
urls = ['https://analytics.usa.gov/', 'https://www.python.org/']
asyncio.run(scrape(urls=urls))